In [1]:
# https://keras.io/examples/generative/wgan_gp/
#

import os
os.environ["CUDA_VISIBLE_DEVICES"] = str(0)
import time

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import *

from tqdm import tqdm
import matplotlib.pyplot as plt
import glob
from sklearn.utils import shuffle

from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy.random import randn
from numpy.random import randint

# example of loading the generator model and generating images
from numpy import asarray
from numpy.random import randn
from numpy.random import randint
from tensorflow.keras.models import load_model
from matplotlib import pyplot


import tensorflow as tf


from tensorflow.keras.datasets.fashion_mnist import load_data
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras import layers as L
from tensorflow.keras import initializers

from skimage import io
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import glob
from sklearn.utils import shuffle

In [18]:
def preprocess_images(images):
  images = (images - 127.5) / 127.5
  return images.astype('float32')

def generator_img(data_list: list):
    counter = 0
    max_counter = len(data_list)
    while True:
        image_s, label_s = data_list[counter]
        image_s = preprocess_images(np.asarray(image_s, dtype=np.float32))
        yield np.expand_dims(image_s, axis=-1), label_s
        counter += 1

        if counter == max_counter:
            counter = 0
            data_list = shuffle(data_list)

In [19]:
IMG_SHAPE = (28, 28, 1)
BATCH_SIZE = 16
N_CLASSES = 10
# Size of the noise vector
noise_dim = 100

SAVE_RESULT = 'exp_result_new_ideas'

train_images_path = []

(x_train, y_train), (_, _) = load_data()

data_list = list(zip(x_train, y_train))
data_list = shuffle(data_list)

def train_gen():
    return generator_img(data_list)

In [4]:
dataset = (
    tf.data.Dataset.from_generator(
        train_gen, 
        output_signature=(
            tf.TensorSpec(shape=IMG_SHAPE, dtype=np.float32),
            tf.TensorSpec(shape=(), dtype=np.int32),
        )
    )
    .shuffle(BATCH_SIZE * 500).batch(BATCH_SIZE).prefetch(6)
)


In [5]:
train_size = len(data_list)

print(f'train: {train_size}')

train: 60000


In [6]:
def init_weights():
    return initializers.RandomNormal(stddev=0.02)

def init_weights():
    return None

In [7]:
class BNInferenceMode(tf.Module):
    def __init__(self, dim, eps=1e-3):
        val = np.ones(dim, dtype='float32')
        self.gamma = tf.Variable(val, name='BN/gamma')
        val = np.zeros(dim, dtype='float32')
        self.beta = tf.Variable(val, name='BN/beta')
        self.eps = eps
    
    def __call__(self, x, training=False):
        mean, var = tf.nn.moments(x, axes=[0, 1, 2], keepdims=True)
        return tf.nn.batch_normalization(
            x=x,
            mean=mean,
            variance=var,
            offset=self.beta,
            scale=self.gamma,
            variance_epsilon=self.eps,
            name='CustomBN'
        )

In [8]:
class GeneratorModel(tf.Module):
    def __init__(self, out_dim, n_classes=10, h_low=3, w_low=3):
        super().__init__()
        # Labels inputs
        self.label_layers_l = self._init_label_input_branch(n_classes, h_low, w_low)
        # Noise inputs
        self.noise_layers_l = self._init_noise_input_branch(h_low, w_low)
        # Merge layer (concat)
        self.merge = Concatenate()
        # Model layers 
        self.model_layers_l = self._init_model_branch(out_dim)
        # Final layer
        self.final_layer = self.model_layers_l[-1]

    def _init_label_input_branch(self, n_classes, h_low, w_low):
        n_nodes = h_low * w_low
        label_layers_l = [
            Embedding(n_classes, 64),
            Dense(n_nodes),
            Reshape((h_low, w_low, 1))
        ]
        return label_layers_l

    def _init_noise_input_branch(self, h_low, w_low):
        # foundation for h_low x w_low image
        n_nodes = 32 * h_low * w_low

        noise_layers_l = [
            Dense(n_nodes),
            LeakyReLU(alpha=0.2),
            Reshape((h_low, w_low, 32))
        ]
        return noise_layers_l

    def _init_model_branch(self, out_dim):
        model_layers_l = [
            Conv2DTranspose(
                128, (3,3), 
                strides=(2,2), padding='same',
                kernel_initializer=init_weights()
            ), # 6
            L.ZeroPadding2D(((1, 0), (1, 0))), # 7
            #BNInferenceMode(128),
            LeakyReLU(alpha=0.2),
            
            Conv2DTranspose(
                64, (3,3), 
                strides=(2,2), padding='same',
                kernel_initializer=init_weights()
            ), # 14
            #BNInferenceMode(128),
            LeakyReLU(alpha=0.2),

            Conv2DTranspose(
                out_dim, (3,3), activation='tanh', 
                strides=(2,2), padding='same',
                kernel_initializer=init_weights()
            ), # 28

        ]
        return model_layers_l
    
    @tf.function
    def __call__(self, label_i, noise_i, training=False):
        # Label branch
        for layer_label_i in self.label_layers_l:
            label_i = layer_label_i(label_i, training=training)

        # Noise branch
        for layer_noise_i in self.noise_layers_l:
            noise_i = layer_noise_i(noise_i, training=training)

        x_t = self.merge([noise_i, label_i])
        # Model branch
        for layer_model_i in self.model_layers_l:
            x_t = layer_model_i(x_t, training=training)
        
        return x_t


In [9]:
class DiscModel(tf.Module):
    def __init__(self, in_shape, out_dim, n_classes=10):
        super().__init__()
        # Labels inputs
        self.label_layers_l = self._init_label_input_branch(in_shape, n_classes)
        # Noise inputs
        self.image_layers_l = self._init_image_input_branch()
        # Merge layer (concat)
        self.merge = Concatenate()
        # Model layers 
        self.model_layers_l = self._init_model_branch(out_dim)
        # Final layer
        self.final_layer = self.model_layers_l[-1]

    def _init_label_input_branch(self, in_shape, n_classes):
        n_nodes = in_shape[0] * in_shape[1] * in_shape[2]
        label_layers_l = [
            # embedding for categorical input
            Embedding(n_classes, 128),
            # scale up to image dimensions with linear activation
            Dense(n_nodes),
            Reshape((in_shape[0], in_shape[1], in_shape[2]))
        ]
        return label_layers_l

    def _init_image_input_branch(self):
        image_layers_l = []
        return image_layers_l

    def _init_model_branch(self, out_dim):
        model_layers_l = [
            Conv2D(
                64, (3,3), 
                strides=(2,2), padding='same', 
                kernel_initializer=init_weights()
            ), # 14
            #BatchNormalization(),
            LeakyReLU(alpha=0.2),

            Conv2D(
                128, (3,3), 
                strides=(2,2), padding='same', 
                kernel_initializer=init_weights()
            ), # 7  
            #BatchNormalization(),
            LeakyReLU(alpha=0.2),

            Conv2D(
                64, (3,3), 
                strides=(2,2), padding='same', 
                kernel_initializer=init_weights()
            ), # 3  
            #BatchNormalization(),
            LeakyReLU(alpha=0.2),
            
            layers.Flatten(),
            layers.Dropout(0.3),
            layers.Dense(out_dim),
        ]
        return model_layers_l
    
    @tf.function
    def __call__(self, label_i, image_i, training=False):
        # Label branch
        for layer_label_i in self.label_layers_l:
            label_i = layer_label_i(label_i, training=training)
        
        # Image branch
        for layer_image_i in self.image_layers_l:
            image_i = layer_image_i(image_i, training=training)
        
        x_t = self.merge([image_i, label_i])
        # Model branch
        for layer_model_i in self.model_layers_l:
            x_t = layer_model_i(x_t, training=training)
        return x_t


In [10]:
d_model = DiscModel(in_shape=IMG_SHAPE, out_dim=1, n_classes=N_CLASSES)

In [11]:
g_model = GeneratorModel(IMG_SHAPE[-1], n_classes=N_CLASSES) 

In [ ]:
#q = g_model.model_layers_l[0]
#q.get_config()

In [13]:
class WGAN(keras.Model):
    def __init__(
        self,
        discriminator,
        generator,
        latent_dim,
        discriminator_extra_steps=3,
        gp_weight=10.0,
    ):
        super(WGAN, self).__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim
        self.d_steps = discriminator_extra_steps
        self.gp_weight = gp_weight

    def compile(self, d_optimizer, g_optimizer, d_loss_fn, g_loss_fn):
        super(WGAN, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.d_loss_fn = d_loss_fn
        self.g_loss_fn = g_loss_fn

    @tf.function
    def gradient_penalty(self, batch_size, real_images, fake_images, real_labels):
        """ Calculates the gradient penalty.

        This loss is calculated on an interpolated image
        and added to the discriminator loss.
        """
        # Get the interpolated image
        alpha = tf.random.normal([batch_size, 1, 1, 1], 0.0, 1.0)
        diff = fake_images - real_images
        interpolated = real_images + alpha * diff

        with tf.GradientTape() as gp_tape:
            gp_tape.watch(interpolated)
            # 1. Get the discriminator output for this interpolated image.
            pred = self.discriminator(label_i=real_labels, image_i=interpolated, training=True)

        # 2. Calculate the gradients w.r.t to this interpolated image.
        grads = gp_tape.gradient(pred, [interpolated, real_labels])[0]
        # 3. Calculate the norm of the gradients.
        norm = tf.sqrt(tf.reduce_sum(tf.square(grads), axis=[1, 2, 3]))
        gp = tf.reduce_mean((norm - 1.0) ** 2)
        return gp

    def train_step(self, real_images, real_labels):
        # Get the batch size
        batch_size = tf.shape(real_images)[0]

        # For each batch, we are going to perform the
        # following steps as laid out in the original paper:
        # 1. Train the generator and get the generator loss
        # 2. Train the discriminator and get the discriminator loss
        # 3. Calculate the gradient penalty
        # 4. Multiply this gradient penalty with a constant weight factor
        # 5. Add the gradient penalty to the discriminator loss
        # 6. Return the generator and discriminator losses as a loss dictionary

        # Train the discriminator first. The original paper recommends training
        # the discriminator for `x` more steps (typically 5) as compared to
        # one step of the generator. Here we will train it for 3 extra steps
        # as compared to 5 to reduce the training time.
        for i in range(self.d_steps):
            d_loss = self._disc_train_step(real_images, real_labels)

        # Train the generator
        # Get the latent vector
        g_loss = self._generator_train_step(batch_size)

        return {"d_loss": d_loss, "g_loss": g_loss}

    @tf.function
    def _generator_train_step(self, batch_size):
        # Train the generator
        # Get the latent vector
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
        random_labels = tf.random.uniform([batch_size], minval=0, maxval=N_CLASSES, dtype=tf.int32)
        with tf.GradientTape() as tape:
            # Generate fake images using the generator
            generated_images = self.generator(label_i=random_labels, noise_i=random_latent_vectors, training=True)
            # Get the discriminator logits for fake images
            gen_img_logits = self.discriminator(label_i=random_labels, image_i=generated_images, training=True)
            # Calculate the generator loss
            g_loss = self.g_loss_fn(gen_img_logits)

        # Get the gradients w.r.t the generator loss
        gen_gradient = tape.gradient(g_loss, self.generator.trainable_variables)
        # Update the weights of the generator using the generator optimizer
        self.g_optimizer.apply_gradients(
            zip(gen_gradient, self.generator.trainable_variables)
        )

        return g_loss

    @tf.function
    def _disc_train_step(self, real_images, real_labels):
        # Get the batch size
        batch_size = tf.shape(real_images)[0]

        random_latent_vectors = tf.random.normal(
            shape=(batch_size, self.latent_dim)
        )
        with tf.GradientTape() as tape:
            # Generate fake images from the latent vector
            fake_images = self.generator(label_i=real_labels, noise_i=random_latent_vectors, training=True)
            # Get the logits for the fake images
            fake_logits = self.discriminator(label_i=real_labels, image_i=fake_images, training=True)
            # Get the logits for the real images
            real_logits = self.discriminator(label_i=real_labels, image_i=real_images, training=True)

            # Calculate the discriminator loss using the fake and real image logits
            d_cost = self.d_loss_fn(real_img=real_logits, fake_img=fake_logits)
            # Calculate the gradient penalty
            gp = self.gradient_penalty(batch_size, real_images, fake_images, real_labels)
            # Add the gradient penalty to the original discriminator loss
            d_loss = d_cost + gp * self.gp_weight

        # Get the gradients w.r.t the discriminator loss
        d_gradient = tape.gradient(d_loss, self.discriminator.trainable_variables)
        # Update the weights of the discriminator using the discriminator optimizer
        self.d_optimizer.apply_gradients(
            zip(d_gradient, self.discriminator.trainable_variables)
        )

        return d_loss



In [14]:
class GANMonitor():
    def __init__(self, model, num_img=100, latent_dim=128, call_sign=None):
        self.num_img = num_img
        self.latent_dim = latent_dim
        self.model = model
        self.call_sign = call_sign

    def on_epoch_end(self, epoch, logs=None, save_path=''):
        n = int(np.sqrt(self.num_img))
        random_latent_vectors = np.random.normal(size=(self.num_img, self.latent_dim))
        random_labels = np.asarray([min(x, N_CLASSES-1)  for _ in range(10) for x in range(10)])
        generated_images = self.model(label_i=random_labels, noise_i=random_latent_vectors)
        # scale from [-1,1] to [0,1]
        generated_images = (generated_images + 1) / 2.0
        self._generate_plot(generated_images, n, os.path.join(save_path, f'{epoch}'))
    
    def _generate_plot(self, examples, n, prefix):
        # plot images
        fig = plt.figure(figsize=(12,12))
        for i in range(n * n):
            # define subplot
            plt.subplot(n, n, 1 + i)
            # turn off axis
            plt.axis('off')
            # plot raw pixel data
            plt.imshow(examples[i])
        #pyplot.show()
        fig.savefig(f'{prefix}_image.png')
        plt.close('all')
    
    def save_model(self, path_save):
        if self.call_sign is None:
            raise ValueError("call sign is not set in monitor class")
        
        tf.saved_model.save(self.model, path_save, signatures=self.call_sign)

In [15]:
import gc

class GCClearCallback:
    def on_epoch_end(self, epoch=0, logs=None):
        gc.collect()
        tf.keras.backend.clear_session()

In [16]:
# Instantiate the optimizer for both networks
# (learning_rate=0.0002, beta_1=0.5 are recommended)
generator_optimizer = keras.optimizers.Adam(
    learning_rate=0.0002, beta_1=0.5, beta_2=0.9
)
discriminator_optimizer = keras.optimizers.Adam(
    learning_rate=0.0002, beta_1=0.5, beta_2=0.9
)

# Define the loss functions for the discriminator,
# which should be (fake_loss - real_loss).
# We will add the gradient penalty later to this loss function.
def discriminator_loss(real_img, fake_img):
    real_loss = tf.reduce_mean(real_img)
    fake_loss = tf.reduce_mean(fake_img)
    return fake_loss - real_loss


# Define the loss functions for the generator.
def generator_loss(fake_img):
    return -tf.reduce_mean(fake_img)


# Set the number of epochs for trainining.
epochs = 20


# Stuf in order to save model
call = g_model.__call__.get_concrete_function(
    tf.TensorSpec((1, 1), tf.int32, name='label'), tf.TensorSpec((1, noise_dim), tf.float32, name='noise')
)

# Instantiate the customer `GANMonitor` Keras callback.
cbk = GANMonitor(
    g_model,
    num_img=100, latent_dim=noise_dim,
    call_sign=call
)
gcclear_call = GCClearCallback
# Instantiate the WGAN model.
wgan = WGAN(
    discriminator=d_model,
    generator=g_model,
    latent_dim=noise_dim,
    discriminator_extra_steps=5, # was 3
)

# Compile the WGAN model.
wgan.compile(
    d_optimizer=discriminator_optimizer,
    g_optimizer=generator_optimizer,
    g_loss_fn=generator_loss,
    d_loss_fn=discriminator_loss,
)


In [ ]:
for ep in range(epochs):
    iteration = train_size // BATCH_SIZE
    save_path = os.path.join(SAVE_RESULT, f'ep_{ep}')
    os.makedirs(save_path, exist_ok=True)
    for i, (real_images, real_labels) in enumerate(dataset.take(iteration)):
        data_losses = wgan.train_step(real_images=real_images, real_labels=real_labels)
        print('>%d, %d/%d, d=%.3f, g=%.3f' %
            (ep+1, i+1, iteration, data_losses['d_loss'], data_losses['g_loss']))
        if i % 20 == 0:
            cbk.on_epoch_end(f'i_{i}_ep_{ep}', save_path=save_path)
        
        if i % 200 == 0:
            cbk.save_model(f'{save_path}/models/i_{i}_ep_{ep}')
    # Clear session
    # Keras iteself has some memory leaks
    # Isshue: https://github.com/tensorflow/tensorflow/issues/31312
    gcclear_call.on_epoch_end(ep)

>1, 1/3750, d=7.813, g=0.001


INFO:tensorflow:Assets written to: exp_result_new_ideas/ep_0/models/i_0_ep_0/assets


INFO:tensorflow:Assets written to: exp_result_new_ideas/ep_0/models/i_0_ep_0/assets


>1, 2/3750, d=6.775, g=0.003
>1, 3/3750, d=5.394, g=-0.007
>1, 4/3750, d=3.205, g=-0.034
>1, 5/3750, d=0.328, g=-0.065
>1, 6/3750, d=-2.924, g=-0.136
>1, 7/3750, d=-7.841, g=-0.328
>1, 8/3750, d=-12.535, g=-0.539
>1, 9/3750, d=-13.395, g=-0.821
>1, 10/3750, d=-15.553, g=-1.151
>1, 11/3750, d=-17.017, g=-1.390
>1, 12/3750, d=-15.151, g=-1.625
>1, 13/3750, d=-21.343, g=-2.072
>1, 14/3750, d=-21.636, g=-2.634
>1, 15/3750, d=-25.020, g=-2.898
>1, 16/3750, d=-23.733, g=-2.804
>1, 17/3750, d=-25.094, g=-3.197
>1, 18/3750, d=-27.494, g=-3.731
>1, 19/3750, d=-24.510, g=-3.825
>1, 20/3750, d=-26.575, g=-3.642
>1, 21/3750, d=-29.549, g=-4.132
>1, 22/3750, d=-29.384, g=-3.525
>1, 23/3750, d=-30.444, g=-3.883
>1, 24/3750, d=-30.001, g=-4.021
>1, 25/3750, d=-29.872, g=-4.486
>1, 26/3750, d=-31.421, g=-4.166
>1, 27/3750, d=-30.277, g=-3.145
>1, 28/3750, d=-29.273, g=-3.704
>1, 29/3750, d=-28.736, g=-3.268
>1, 30/3750, d=-29.713, g=-3.766
>1, 31/3750, d=-31.558, g=-3.639
>1, 32/3750, d=-31.017, g=-2.

INFO:tensorflow:Assets written to: exp_result_new_ideas/ep_0/models/i_200_ep_0/assets


...

INFO:tensorflow:Assets written to: exp_result_new_ideas/ep_0/models/i_600_ep_0/assets


>1, 602/3750, d=-7.580, g=26.165
>1, 603/3750, d=-12.684, g=29.446
>1, 604/3750, d=-12.597, g=27.055
>1, 605/3750, d=-13.330, g=26.971
>1, 606/3750, d=-15.794, g=24.439
>1, 607/3750, d=-14.165, g=26.441
>1, 608/3750, d=-7.135, g=29.431
>1, 609/3750, d=-13.190, g=30.240
>1, 610/3750, d=-13.489, g=28.021
>1, 611/3750, d=-14.917, g=24.403
>1, 612/3750, d=-11.104, g=27.925
>1, 613/3750, d=-17.140, g=28.240
>1, 614/3750, d=-13.941, g=27.490
>1, 615/3750, d=-19.752, g=28.879
>1, 616/3750, d=-14.894, g=28.835
>1, 617/3750, d=-15.633, g=27.983
>1, 618/3750, d=-12.013, g=26.856
>1, 619/3750, d=-11.717, g=29.784
>1, 620/3750, d=-16.977, g=35.196
>1, 621/3750, d=-16.472, g=30.556
>1, 622/3750, d=-13.858, g=31.810
>1, 623/3750, d=-12.056, g=33.752
>1, 624/3750, d=-11.172, g=40.836
>1, 625/3750, d=-15.073, g=33.741
>1, 626/3750, d=-17.329, g=33.368
>1, 627/3750, d=-20.968, g=32.969
>1, 628/3750, d=-13.425, g=35.938
>1, 629/3750, d=-12.028, g=31.159
>1, 630/3750, d=-19.550, g=36.320
>1, 631/3750, d=

INFO:tensorflow:Assets written to: exp_result_new_ideas/ep_0/models/i_800_ep_0/assets


INFO:tensorflow:Assets written to: exp_result_new_ideas/ep_0/models/i_800_ep_0/assets


>1, 802/3750, d=-10.234, g=10.092
>1, 803/3750, d=-7.771, g=8.539
>1, 804/3750, d=-14.287, g=8.242
>1, 805/3750, d=-8.888, g=-0.713
>1, 806/3750, d=-15.629, g=5.540
>1, 807/3750, d=-12.497, g=1.668
>1, 808/3750, d=-10.068, g=3.396
>1, 809/3750, d=-7.633, g=0.064
>1, 810/3750, d=-10.253, g=-5.111
>1, 811/3750, d=-10.545, g=-2.430
>1, 812/3750, d=-16.692, g=1.345
>1, 813/3750, d=-17.611, g=5.110
>1, 814/3750, d=-11.602, g=-4.936
>1, 815/3750, d=-9.185, g=-0.626
>1, 816/3750, d=-8.947, g=2.358
>1, 817/3750, d=-11.902, g=4.934
>1, 818/3750, d=-14.608, g=4.951
>1, 819/3750, d=-10.466, g=17.947
>1, 820/3750, d=-11.757, g=15.004
>1, 821/3750, d=-11.931, g=17.855
>1, 822/3750, d=-16.771, g=16.776
>1, 823/3750, d=-16.893, g=11.598
>1, 824/3750, d=-13.562, g=16.926
>1, 825/3750, d=-8.994, g=10.880
>1, 826/3750, d=-12.919, g=14.258
>1, 827/3750, d=-14.322, g=14.317
>1, 828/3750, d=-11.559, g=10.621
>1, 829/3750, d=-10.711, g=17.632
>1, 830/3750, d=-12.371, g=7.889
>1, 831/3750, d=-13.440, g=5.050

In [ ]:
call = g_model.__call__.get_concrete_function(
    tf.TensorSpec((1, 1), tf.int32, name='label'), tf.TensorSpec((1, noise_dim), tf.float32, name='noise')
)

In [ ]:
tf.saved_model.save(g_model, f'{SAVE_RESULT}/models/test_save', signatures=call)

In [ ]:
model = tf.saved_model.load(f'{SAVE_RESULT}/models/test_save')

In [ ]:
random_d = np.random.randn(1, noise_dim).astype(np.float32)
random_l = np.random.randint(low=0, high=N_CLASSES, size=(1, 1)).astype(np.int32)

In [ ]:
output_np = model(random_l, random_d)

In [ ]:
plt.imshow((output_np[0]+1) / 2.0)

In [ ]:
w = g_model.label_layers_l[0].get_weights()[0]

In [ ]:
w /= np.expand_dims(np.sqrt(np.sum(w * w, axis=1)), axis=1)
h = np.dot(w, w.T)

In [ ]:
import seaborn as sns

In [ ]:
sns.heatmap(h)